In [189]:
#Cargamos librerias 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [313]:
#Carga desde un archivo .csv sin indice y eliminar filas innecesarias
mexico=pd.read_csv("mexico_limpio.csv")
mexico.head(5)

,Unnamed: 0,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,...,room_type,bathrooms_text,amenities,calendar_updated,has_availability,calendar_last_scraped,first_review,last_review,license,instant_bookable
0,0,3.0,6.0,19.42063,-99.16586,2.0,1.0,1.0,2.0,1652.0,...,Entire home/apt,1 bath,"[""40 inch HDTV with Amazon Prime Video, Netfli...",nulos,t,2024-06-28,2013-03-30,2024-04-17,nulos,f
1,1,1.0,1.0,19.38283,-99.20000,2.0,1.0,1.0,1.0,3663.0,...,Entire home/apt,1 bath,"[""Hot water"", ""Washer"", ""Bed linens"", ""Wifi"", ...",nulos,t,2024-06-27,2013-03-30,2024-04-17,nulos,f
2,2,17.0,17.0,19.41162,-99.17794,3.1,1.4,5.0,1.8,1314.4,...,Entire home/apt,5.5 baths,"[""Wifi"", ""Coffee maker: Nespresso"", ""Host gree...",nulos,t,2024-06-28,2011-11-09,2023-03-26,nulos,f
3,3,1.0,4.0,19.43956,-99.17263,4.0,1.0,2.0,3.0,814.0,...,Entire home/apt,1 bath,"[""Wifi"", ""Host greets you"", ""Coffee maker"", ""S...",nulos,t,2024-06-28,2013-05-16,2024-05-21,nulos,t
4,4,3.0,4.0,19.41152,-99.16857,4.0,1.0,2.0,2.0,1923.0,...,Entire home/apt,1 bath,"[""Wifi"", ""Coffee maker"", ""Sound system"", ""Clot...",nulos,t,2024-06-28,2011-11-17,2024-05-05,nulos,f


In [191]:
#Rellenamos valores nulos
mexico= mexico.fillna(method='bfill')
mexicof= mexico.fillna(method='ffill')

/var/folders/wy/qgq0rkx14f3dj_29_v68hsvc0000gn/T/ipykernel_12274/4050697152.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexico= mexico.fillna(method='bfill')
/var/folders/wy/qgq0rkx14f3dj_29_v68hsvc0000gn/T/ipykernel_12274/4050697152.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexicof= mexico.fillna(method='ffill')


In [192]:
mexico['price'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 26536 entries, 0 to 26535
Series name: price
Non-Null Count  Dtype  
--------------  -----  
26536 non-null  float64
dtypes: float64(1)
memory usage: 207.4 KB


**1 categorica**

In [193]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=mexico['price'].max()
Min=mexico['price'].min()
Limites= [Min, Max]
Limites

[105.0, 13283.0]

In [194]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(105.0, 13283.0, 3)
intervalos

array([  105.,  6694., 13283.])

In [195]:
#Creamos las categorías 
categorias= ["cheap", "expensive"]

In [196]:
#Finalmente creamos las categorías en la columna numérica
mexico['price']=pd.cut(x= mexico['price'], bins=intervalos, labels= categorias )

In [197]:
#Rellenamos valores nulos
mexico= mexico.fillna(method='bfill')
mexicof= mexico.fillna(method='ffill')

/var/folders/wy/qgq0rkx14f3dj_29_v68hsvc0000gn/T/ipykernel_12274/4050697152.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexico= mexico.fillna(method='bfill')
/var/folders/wy/qgq0rkx14f3dj_29_v68hsvc0000gn/T/ipykernel_12274/4050697152.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexicof= mexico.fillna(method='ffill')


In [198]:
mexico['price'].isnull().sum()

0

In [199]:
vars_Indep = mexico[['bedrooms']]
#'bathrooms'
vars_Dep = mexico['price']
#'accommodates', 'host_response_rate', 'host_acceptance_rate', 'host_total_listings_count', 'reviews_per_month'

In [200]:
#Redefinimos las variables 
x= vars_Indep
y= vars_Dep

In [201]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba 
X_test, X_train, Y_test, Y_train= train_test_split(x,y, test_size=0.3, random_state=None)

In [202]:
escalar = StandardScaler()

In [203]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [204]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [205]:
#Entrenamos el modelo 
algoritmo.fit(X_train, Y_train)

LogisticRegression()

In [206]:
#realizamos una prediccion 
y_pred = algoritmo.predict(X_test)
y_pred

array(['cheap', 'cheap', 'cheap', ..., 'cheap', 'cheap', 'cheap'],
      dtype=object)

In [207]:
#Verifico la matriz de confusion 
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test, y_pred)
print('Matriz de Confusion')
print(matriz)

Matriz de Confusion
[[18380     0]
 [  195     0]]


In [208]:
#CAlculamos la prediccion del modelo 
from sklearn.metrics import precision_score

precision = precision_score(Y_test, y_pred, average="binary", pos_label="cheap")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.9895020188425303


In [209]:
#Calculamos la exactitud del modelo 
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(Y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9895020188425303


In [210]:
#calculamos la sensibilidad del modelo 
from sklearn.metrics import recall_score

sensibilidad = recall_score(Y_test, y_pred, average="binary", pos_label="cheap")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


**2 Categoricas**

In [211]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=mexico['accommodates'].max()
Min=mexico['accommodates'].min()
Limites= [Min, Max]
Limites

[1.0, 10.0]

In [212]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos1=np.linspace(0.99, 10.1, 3)
intervalos1

array([ 0.99 ,  5.545, 10.1  ])

In [213]:
#Creamos las categorías 
categorias1= ["pocos", "manada"]

In [214]:
mexico['accommodates']

0        2.0
1        2.0
2        3.1
3        4.0
4        4.0
        ... 
26531    3.1
26532    3.0
26533    3.0
26534    4.0
26535    4.0
Name: accommodates, Length: 26536, dtype: float64

In [215]:
#Finalmente creamos las categorías en la columna numérica
mexico['accommodates']=pd.cut(x= mexico['accommodates'], bins=intervalos1, labels= categorias1 )

In [216]:
#Rellenamos valores nulos
mexico= mexico.fillna(method='bfill')
mexicof= mexico.fillna(method='ffill')

/var/folders/wy/qgq0rkx14f3dj_29_v68hsvc0000gn/T/ipykernel_12274/4050697152.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexico= mexico.fillna(method='bfill')
/var/folders/wy/qgq0rkx14f3dj_29_v68hsvc0000gn/T/ipykernel_12274/4050697152.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexicof= mexico.fillna(method='ffill')


In [217]:
mexico['accommodates'].isnull().sum()

0

In [219]:
vars_Indep_1 = variablesselec_h[['bedrooms']]
#'bathrooms'
vars_Dep_1 = variablesselec_h['accommodates']
#'accommodates', 'host_response_rate', 'host_acceptance_rate', 'host_total_listings_count', 'reviews_per_month'

In [220]:
#Redefinimos las variables 
x= vars_Indep_1
y= vars_Dep_1

In [221]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba 
X_test1, X_train1, Y_test1, Y_train1= train_test_split(x,y, test_size=0.3, random_state=None)

In [222]:
escalar1 = StandardScaler()

In [223]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos 
X_train1 = escalar.fit_transform(X_train1)
X_test1 = escalar.transform(X_test1)

In [224]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo1 = LogisticRegression()

In [227]:
#Entrenamos el modelo 
algoritmo1.fit(X_train1, Y_train1)

LogisticRegression()

In [228]:
#realizamos una prediccion 
y_pred1 = algoritmo1.predict(X_test1)
y_pred1

array(['pocos', 'pocos', 'pocos', ..., 'pocos', 'pocos', 'pocos'],
      dtype=object)

In [229]:
#Verifico la matriz de confusion 
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test1, y_pred1)
print('Matriz de Confusion')
print(matriz)

Matriz de Confusion
[[ 1308   833]
 [  577 15857]]


In [230]:
#CAlculamos la prediccion del modelo 
from sklearn.metrics import precision_score

precision = precision_score(Y_test1, y_pred1, average="binary", pos_label="pocos")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.9500898741761534


In [231]:
#Calculamos la exactitud del modelo 
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(Y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9240915208613728


In [232]:
#calculamos la sensibilidad del modelo 
from sklearn.metrics import recall_score

sensibilidad = recall_score(Y_test1, y_pred1, average="binary", pos_label="pocos")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9648898624802239


**3 Categoricas**

In [263]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=mexico['beds'].max()
Min=mexico['beds'].min()
Limites= [Min, Max]
Limites

[0.0, 7.0]

In [264]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos2=np.linspace(-0.1, 7.1, 3)
intervalos2


array([-0.1,  3.5,  7.1])

In [265]:
#Creamos las categorías 
categorias2= ["dormilones", "activos"]

In [266]:
mexico['beds']

0        2.0
1        1.0
2        1.8
3        3.0
4        2.0
        ... 
26531    7.0
26532    2.0
26533    1.0
26534    2.0
26535    2.0
Name: beds, Length: 26536, dtype: float64

In [267]:
#Finalmente creamos las categorías en la columna numérica
mexico['beds']=pd.cut(x= mexico['beds'], bins=intervalos2, labels= categorias2 )


In [268]:
#Rellenamos valores nulos
mexico= mexico.fillna(method='bfill')
mexicof= mexico.fillna(method='ffill')


/var/folders/wy/qgq0rkx14f3dj_29_v68hsvc0000gn/T/ipykernel_12274/3541779558.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexico= mexico.fillna(method='bfill')
/var/folders/wy/qgq0rkx14f3dj_29_v68hsvc0000gn/T/ipykernel_12274/3541779558.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexicof= mexico.fillna(method='ffill')


In [269]:
mexico['beds'].isnull().sum()

0

In [278]:
vars_Indep_2 = mexico[['bedrooms']]

vars_Dep_2 = mexico['beds']


In [279]:
#Redefinimos las variables 
x= vars_Indep_2
y= vars_Dep_2


In [280]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba 
X_test2, X_train2, Y_test2, Y_train2= train_test_split(x,y, test_size=0.3, random_state=None)


In [281]:
escalar2 = StandardScaler()

In [282]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos 
X_train2 = escalar.fit_transform(X_train2)
X_test2 = escalar.transform(X_test2)


In [283]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo2 = LogisticRegression()


In [284]:
#Entrenamos el modelo 
algoritmo2.fit(X_train2, Y_train2)


LogisticRegression()

In [285]:
#realizamos una prediccion 
y_pred2 = algoritmo2.predict(X_test2)
y_pred2


array(['dormilones', 'dormilones', 'dormilones', ..., 'dormilones',
       'dormilones', 'dormilones'], dtype=object)

In [286]:
#Verifico la matriz de confusion 
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test2, y_pred2)
print('Matriz de Confusion')
print(matriz)


Matriz de Confusion
[[  287  1349]
 [  155 16784]]


In [287]:
#CAlculamos la prediccion del modelo 
from sklearn.metrics import precision_score
precision = precision_score(Y_test2, y_pred2, average="binary", pos_label="dormilones")
print('Precision del modelo:')
print(precision)


Precision del modelo:
0.9256052500965092


In [288]:
#Calculamos la exactitud del modelo 
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test2, y_pred2)
print('Exactitud del modelo:')
print(exactitud)


Exactitud del modelo:
0.9190309555854643


In [289]:
#calculamos la sensibilidad del modelo 
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test2, y_pred2, average="binary", pos_label="dormilones")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.9908495188617982


**4 categoricas**

In [292]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=mexico['bathrooms'].max()
Min=mexico['bathrooms'].min()
Limites= [Min, Max]
Limites


[0.0, 4.5]

In [293]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos3=np.linspace(-0.1, 4.6, 3)
intervalos3


array([-0.1 ,  2.25,  4.6 ])

In [294]:
#Creamos las categorías 
categorias3= ["extreñidos", "surrones"]


In [295]:
mexico['bathrooms']

0        1.0
1        1.0
2        1.4
3        1.0
4        1.0
        ... 
26531    4.5
26532    1.0
26533    1.0
26534    1.5
26535    1.0
Name: bathrooms, Length: 26536, dtype: float64

In [296]:
#Finalmente creamos las categorías en la columna numérica
mexico['bathrooms']=pd.cut(x= mexico['bathrooms'], bins=intervalos3, labels= categorias3 )


In [297]:
#Rellenamos valores nulos
mexico= mexico.fillna(method='bfill')
mexicof= mexico.fillna(method='ffill')


/var/folders/wy/qgq0rkx14f3dj_29_v68hsvc0000gn/T/ipykernel_12274/3541779558.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexico= mexico.fillna(method='bfill')
/var/folders/wy/qgq0rkx14f3dj_29_v68hsvc0000gn/T/ipykernel_12274/3541779558.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexicof= mexico.fillna(method='ffill')


In [298]:
mexico['bathrooms'].isnull().sum()

0

In [300]:
vars_Indep_3 = mexico[['bedrooms']]

vars_Dep_3 = mexico['bathrooms']


In [301]:
#Redefinimos las variables 
x= vars_Indep_3
y= vars_Dep_3


In [302]:

#dividimos el conjunto de datos en la parte de entrenamiento y pruba 
X_test3, X_train3, Y_test3, Y_train3= train_test_split(x,y, test_size=0.3, random_state=None)


In [303]:
escalar3 = StandardScaler()

In [304]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos 
X_train3 = escalar.fit_transform(X_train3)
X_test3 = escalar.transform(X_test3)


In [305]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo3 = LogisticRegression()


In [306]:
#Entrenamos el modelo 
algoritmo3.fit(X_train3, Y_train3)


LogisticRegression()

In [307]:
#realizamos una prediccion 
y_pred3 = algoritmo3.predict(X_test3)
y_pred3


array(['extreñidos', 'extreñidos', 'extreñidos', ..., 'extreñidos',
       'extreñidos', 'extreñidos'], dtype=object)

In [308]:
#Verifico la matriz de confusion 
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test3, y_pred3)
print('Matriz de Confusion')
print(matriz)


Matriz de Confusion
[[16801   159]
 [ 1355   260]]


In [310]:
#CAlculamos la prediccion del modelo 
from sklearn.metrics import precision_score
precision = precision_score(Y_test3, y_pred3, average="binary", pos_label="extreñidos")
print('Precision del modelo:')
print(precision)


Precision del modelo:
0.9253690240141


In [311]:
#Calculamos la exactitud del modelo 
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test3, y_pred3)
print('Exactitud del modelo:')
print(exactitud)


Exactitud del modelo:
0.918492597577389


In [312]:
#calculamos la sensibilidad del modelo 
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test3, y_pred3, average="binary", pos_label="extreñidos")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.990625


**5 Categoria**

In [317]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=mexico['host_total_listings_count'].max()
Min=mexico['host_total_listings_count'].min()
Limites= [Min, Max]
Limites


[1.0, 442.0]

In [316]:
mexico['host_total_listings_count']

0          6.0
1          1.0
2         17.0
3          4.0
4          4.0
         ...  
26531     15.0
26532     51.0
26533     15.0
26534      3.0
26535    256.0
Name: host_total_listings_count, Length: 26536, dtype: float64

In [319]:
#Categorización de variables
#Declaramos 2 intervalos 
intervalos4=np.linspace(0.99, 442.1, 3)
intervalos4


array([  0.99 , 221.545, 442.1  ])

In [320]:
#Creamos las categorías 
categorias4= ["pocas", "muchas"]


In [321]:
mexico['host_total_listings_count']

0          6.0
1          1.0
2         17.0
3          4.0
4          4.0
         ...  
26531     15.0
26532     51.0
26533     15.0
26534      3.0
26535    256.0
Name: host_total_listings_count, Length: 26536, dtype: float64

In [322]:
#Finalmente creamos las categorías en la columna numérica
mexico['host_total_listings_count']=pd.cut(x= mexico['host_total_listings_count'], bins=intervalos4, labels= categorias4 )


In [323]:
#Rellenamos valores nulos
mexico= mexico.fillna(method='bfill')
mexicof= mexico.fillna(method='ffill')


/var/folders/wy/qgq0rkx14f3dj_29_v68hsvc0000gn/T/ipykernel_12274/3541779558.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexico= mexico.fillna(method='bfill')
/var/folders/wy/qgq0rkx14f3dj_29_v68hsvc0000gn/T/ipykernel_12274/3541779558.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mexicof= mexico.fillna(method='ffill')


In [324]:
mexico['host_total_listings_count'].isnull().sum()

0

In [330]:
vars_Indep_4 = mexico[['bedrooms']]

vars_Dep_4 = mexico['host_total_listings_count']


In [332]:
#Redefinimos las variables 
x= vars_Indep_4
y= vars_Dep_4


In [333]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba 
X_test4, X_train4, Y_test4, Y_train4= train_test_split(x,y, test_size=0.3, random_state=None)


In [334]:
escalar4 = StandardScaler()

In [335]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos 
X_train4 = escalar.fit_transform(X_train4)
X_test4 = escalar.transform(X_test4)


In [336]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo4 = LogisticRegression()


In [337]:
#Entrenamos el modelo 
algoritmo4.fit(X_train4, Y_train4)


LogisticRegression()

In [338]:
#realizamos una prediccion 
y_pred4 = algoritmo4.predict(X_test4)
y_pred4


array(['pocas', 'pocas', 'pocas', ..., 'pocas', 'pocas', 'pocas'],
      dtype=object)

In [339]:
#Verifico la matriz de confusion 
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test4, y_pred4)
print('Matriz de Confusion')
print(matriz)


Matriz de Confusion
[[    0   460]
 [    0 18115]]


In [340]:
#CAlculamos la prediccion del modelo 
from sklearn.metrics import precision_score
precision = precision_score(Y_test4, y_pred4, average="binary", pos_label="pocas")
print('Precision del modelo:')
print(precision)


Precision del modelo:
0.9752355316285329


In [341]:
#Calculamos la exactitud del modelo 
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test4, y_pred4)
print('Exactitud del modelo:')
print(exactitud)


Exactitud del modelo:
0.9752355316285329


In [342]:
#calculamos la sensibilidad del modelo 
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test4, y_pred4, average="binary", pos_label="pocas")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
1.0


**FIN**